## Make a simple synthetic dataset to test hierarchical merge in FOF algorithm

#### the idea is this: 

* after the local FOF stage, each partition reports the particles it holds in the overlap region
* do a reduceByKey or treeAggregate of some sort to collect the groups belonging to the same particles
* produce a mapping of $G -> G_1$ and distribute to all hosts in form of broadcast lookup table

In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

import sys
sys.setrecursionlimit(sys.getrecursionlimit()*10)
import matplotlib.patches as patches

plt.style.use('bmh')

In [2]:
import findspark
findspark.init()

In [3]:
def plot_rectangle(rec, ax=None):
    if ax is None: 
        ax = plt.subplot(aspect='equal')
    
    if isinstance(rec, (list, tuple)):
        for r in rec: 
            plot_rectangle(r,ax)
    
    else:
        size = (rec.maxes-rec.mins)
        ax.add_patch(patches.Rectangle(rec.mins, size[0], size[1], fill=False, zorder=-1))
    plt.draw()
    plt.show()

## Start Spark

In [4]:
import os
os.environ['SPARK_CONF_DIR'] = './conf'
os.environ['SPARK_DRIVER_MEMORY'] = '8G'

In [5]:
import pyspark
from pyspark import SparkContext, SparkConf
import pynbody
import sparkhpc

In [6]:
sj = sparkhpc.sparkjob.LSFSparkJob(ncores=6, memory=8500, walltime='4:00', jobid=32588668)
#sj.wait_to_start()

In [7]:
conf = SparkConf()

In [8]:
conf.set('spark.python.profile', 'true')
conf.set('spark.executor.memory', '8G')
conf.set('spark.driver.memory', '8G')
conf.set('spark.driver.maxResultSize', '5G')

In [85]:
sc = SparkContext(master=sj.master_url(), conf=conf)

In [10]:
sj.master_ui()

'http://10.205.7.45:8080'

### Make the base RDD

In [86]:
def convert_to_fof_particle(s): 
    p_arr = np.frombuffer(s, pdt_tipsy)
    
    new_arr = np.zeros(len(p_arr), dtype=pdt)
    new_arr['pos'] = p_arr['pos']
    
    return new_arr

In [87]:
def get_minmax(index, i): 
    mins = []
    maxs = []
    for arr in i: 
        mins.append(arr['pos'].min(axis=0))
        maxs.append(arr['pos'].max(axis=0))
    
    yield (index, (np.concatenate(mins).reshape(len(mins),3).min(axis=0), np.concatenate(maxs).reshape(len(mins),3).max(axis=0)))

In [88]:
from pyspark.accumulators import AccumulatorParam

class dictAdd(AccumulatorParam):
    def zero(self, value):
        return {i:0 for i in range(len(value))}
    def addInPlace(self, val1, val2): 
        for k, v in val2.iteritems(): 
            val1[k] += v
        return val1

In [89]:
from spark_fof.spark_fof_c import pdt
from spark_fof.spark_fof import partition_helper

In [90]:
# function to map from file block numbers to domain bin
N = 62
map_file_to_domain = lambda (x,y,z): (x-1) + (y-1)*N + (z-1)*N*N

pdt_lc = np.dtype([('pos', 'f4', 3),('vel', 'f4', 3)])

import re
import time

get_block_ids = re.compile('blk\.(\d+)\.(\d+)\.(\d+)i')

def make_lc_rdd(sc, path):
    from glob import glob

    def set_particle_IDs_partition(index, iterator): 
        p_counts = partition_counts.value
        local_index = 0
        start_index = sum([p_counts[i] for i in range(index)])
        for arr in iterator:
            arr['iOrder'] = range(start_index + local_index, start_index + local_index + len(arr))
            local_index += len(arr)
            yield arr
    
    def read_file(index, i, chunksize=102400): 
        for part,filename in i:
            timein = time.time()
            with open(filename,'rb') as f: 
                header = f.read(62500)
                while True:
                    chunk = f.read(chunksize*24)
                    if len(chunk): 
                        p_arr = np.frombuffer(chunk, pdt_lc)
                        new_arr = np.zeros(len(p_arr), dtype=pdt)
                        new_arr['pos'] = p_arr['pos']
                        npart_acc.add({index: len(p_arr)})
                        yield new_arr
                    else: 
                        print 'reading %s took %d seconds in partition %d'%(filename, time.time()-timein, index)
                        break
                    
    files = glob('/cluster/home/roskarr/scratch/euclid/2Tlc-final/*/*')

    ids = map(lambda x: tuple(map(int, get_block_ids.findall(x)[0])), files)
    ids_map = {x:i for i,x in enumerate(ids)}
    ids_map_b = sc.broadcast(ids_map)
    nfiles = len(files)
    
    print 'Number of input files: ', nfiles
    
    # set the partition count accumulator
    npart_acc = sc.accumulator({i:0 for i in range(nfiles)}, dictAdd())
    
    rec_rdd = (sc.parallelize(zip(range(nfiles),files), numSlices=sc.defaultParallelism)
                 .partitionBy(sc.defaultParallelism)
                 .mapPartitionsWithIndex(read_file, preservesPartitioning=True))
    
    rec_rdd.count()
    partition_counts = sc.broadcast(npart_acc.value)
    
    return partition_counts, ids_map, rec_rdd.mapPartitionsWithIndex(set_particle_IDs_partition, 
                                                                     preservesPartitioning=True)

In [91]:
%%time 
p_counts, ids_map, p_rdd = make_lc_rdd(sc, '2Tlc-final/')
#p_rdd.cache()
#p_rdd.count()


Number of input files:  6
CPU times: user 20 ms, sys: 5 ms, total: 25 ms
Wall time: 13.4 s


In [92]:
p_rdd.cache()

PythonRDD[6] at RDD at PythonRDD.scala:48

In [18]:
#%time limits = p_rdd.mapPartitionsWithIndex(get_minmax).collect()

### Preliminaries, setting the domain size and global grid mapping

In [19]:
#diff = (limits[0][1][1] - limits[0][1][0])[0]

In [93]:
#diff = (limits[0][1][1] - limits[0][1][0])[0]
diff = np.float32(0.033068776)
global_min = -31*diff
global_max = 31*diff

diff, global_min, global_max

(0.033068776, -1.0251320600509644, 1.0251320600509644)

In [94]:
dom_maxs = np.array([global_max]*3, dtype=np.float64)
dom_mins = np.array([global_min]*3, dtype=np.float64)

#tau = diff*5./125.
tau = 0.2/12600
buffer_tau = diff*5./150.

global_to_cutout_map = {}
for k,v in ids_map.iteritems(): 
    global_to_cutout_map[map_file_to_domain(k)] = v

### Run FOF and Merge groups

In [95]:
import spark_fof

In [96]:
reload(spark_fof.spark_fof)
reload(spark_fof.spark_fof_c)

fof_analyzer = spark_fof.spark_fof.FOFAnalyzer(sc, p_rdd, 64, 62, tau, dom_mins, dom_maxs, Npartitions=6, buffer_tau=buffer_tau)

In [97]:
### THIS MUST BE FIXED IN THE MAIN CODE
fof_analyzer.global_to_local_map = global_to_cutout_map

In [98]:
def ghost_counts(index, iterator): 
    nghosts = 0
    nghosts_copy = 0
    nother = 0
    for arr in iterator: 
        nghosts += len(np.where(arr['is_ghost']==1)[0])
        nghosts_copy += len(np.where(arr['is_ghost']==2)[0])
        nother += len(np.where(arr['is_ghost']==0)[0])
    yield index,nother,nghosts,nghosts_copy

In [99]:
fof_analyzer.partitioned_rdd.mapPartitionsWithIndex(ghost_counts).collect()

[(0, 68925308, 6515664, 3505),
 (1, 66661075, 7944053, 33062),
 (2, 62581986, 6012280, 31711),
 (3, 59395506, 7036506, 20),
 (4, 67257547, 7985552, 4),
 (5, 66665360, 6877971, 18)]

In [26]:
%%time 
fof_rdd = fof_analyzer.fof_rdd
fof_rdd.count()

CPU times: user 10.6 s, sys: 336 ms, total: 10.9 s
Wall time: 4min 43s


In [31]:
from spark_fof.spark_fof import get_bin

domain_containers = fof_analyzer.domain_containers

mins = fof_analyzer.mins
maxs = fof_analyzer.maxs
nbins = fof_analyzer.nBins

global_to_local_map = fof_analyzer.global_to_local_map
gl_to_loc_map_b = sc.broadcast(global_to_local_map)

def set_bin(pos): 
    pos = np.array(pos, dtype=np.float32)
    gl_to_loc_map = gl_to_loc_map_b.value
    
    mybin = get_bin(np.array(pos, dtype=np.float32),nbins,mins,maxs)
    
    
set_bin = lambda pos: gl_to_loc_map_b.value[get_bin(np.array(pos, dtype=np.float32),nbins,mins,maxs)]

def get_bin2(particles):
    for p in particles:
        b = get_bin(p['pos'],nbins,mins,maxs)
        if b in gl_to_loc_map_b.value:
            yield (gl_to_loc_map_b.value[b], p)
    

AttributeError: FOFAnalyzer instance has no attribute 'global_to_local_map'

In [ ]:
groups_map = (fof_rdd.flatMap(lambda p: p[np.where(p['is_ghost'])[0]])
                     .mapPartitions(get_bin2)
                     .partitionBy(6)).cache()
#                       .map(lambda (partition_id, (particle_id,group_id)): (particle_id,group_id), 
#                            preservesPartitioning=True))
#                      .aggregateByKey([], lambda l, g: l + [g], lambda a, b: sorted(a + b), 
#                                      partitionFunc=set_bin))
#                      .values()
#                      .flatMap(lambda gs: [(g, gs[0]) for g in gs[1:]])).collect()

In [ ]:
groups_map.first()

In [27]:
x = fof_rdd.flatMap(lambda p: p[np.where(p['is_ghost']>0)[0]]).map(lambda p: (get_bin(p['pos'],nbins,mins,maxs), p))

In [39]:
fof_rdd.flatMap(lambda p: p[np.where(p['is_ghost']==2)[0]])\
       .mapPartitions(get_bin2).take(20)

[(78680421, 1),
 (78680422, 1),
 (78680429, 1),
 (78680434, 1),
 (78680538, 1),
 (78680710, 1),
 (78680712, 1),
 (78787492, 1),
 (78787559, 1),
 (78787861, 1),
 (78787961, 1),
 (78787964, 1),
 (78787968, 1),
 (78788236, 1),
 (78788237, 1),
 (78788238, 1),
 (78788240, 1),
 (78788244, 1),
 (78788595, 1),
 (78788596, 1)]

In [28]:
bad = x.filter(lambda (binid,p): binid not in gl_to_loc_map_b.value).collect()

In [34]:
fof_rdd.flatMap(lambda x: x[np.where(x['is_ghost']==2)[0]]).take(20)

[([-0.06573289632797241, -0.06575457006692886, -0.06612294912338257], 1, 0, 12),
 ([-0.06573054194450378, -0.06575792282819748, -0.06612083315849304], 1, 1, 12),
 ([-0.0657329186797142, -0.06576550006866455, -0.06612081825733185], 1, 2, 12),
 ([-0.06572426110506058, -0.06575217097997665, -0.0661124736070633], 1, 3, 12),
 ([-0.06572258472442627, -0.06576202064752579, -0.06610748916864395], 1, 4, 12),
 ([-0.06572159379720688, -0.06575781106948853, -0.06611243635416031], 1, 5, 12),
 ([-0.06577809154987335, -0.06563536077737808, -0.06611169129610062], 1, 6, 22),
 ([-0.06577938795089722, -0.06562937796115875, -0.06611119955778122], 1, 7, 22),
 ([-0.0657726302742958, -0.06563825905323029, -0.06612486392259598], 1, 8, 22),
 ([-0.0657418817281723, -0.06561712920665741, -0.0660964697599411], 1, 9, 22),
 ([-0.06573700904846191, -0.0656098797917366, -0.06609456986188889], 1, 10, 22),
 ([-0.06577926874160767, -0.06562548875808716, -0.06606430560350418], 1, 11, 22),
 ([-0.06577067077159882, -0.0656

In [44]:
fof_analyzer.partitioned_rdd.flatMap(lambda p: p[np.where(p['iOrder']==4721)]).take(2)

[([-0.0655440017580986, -0.06568702310323715, -0.06613755971193314], 2, 4721, 0)]

In [39]:
fof_rdd.flatMap(lambda x: x[np.where(x['iOrder']==78788236)]).take(2)

[([-0.06543324887752533, -0.06370767951011658, -0.033066850155591965], 2, 78788236, 10066285),
 ([-0.06543324887752533, -0.06370767951011658, -0.033066850155591965], 1, 78788236, 4294980728)]

In [45]:
ids_map

{(30, 30, 30): 0,
 (30, 30, 31): 1,
 (30, 30, 32): 2,
 (30, 31, 30): 3,
 (30, 31, 31): 4,
 (30, 31, 32): 5,
 (30, 32, 30): 6,
 (30, 32, 31): 7,
 (30, 32, 32): 8,
 (31, 30, 30): 9,
 (31, 30, 31): 10,
 (31, 30, 32): 11,
 (31, 31, 30): 12,
 (31, 31, 31): 13,
 (31, 31, 32): 14,
 (31, 32, 30): 15,
 (31, 32, 31): 16,
 (31, 32, 32): 17,
 (32, 30, 30): 18,
 (32, 30, 31): 19,
 (32, 30, 32): 20,
 (32, 31, 30): 21,
 (32, 31, 31): 22,
 (32, 31, 32): 23,
 (32, 32, 30): 24,
 (32, 32, 31): 25,
 (32, 32, 32): 26}

In [30]:
global_to_local_map

{113303: 0,
 113304: 9,
 113305: 18,
 113365: 3,
 113366: 12,
 113367: 21,
 113427: 6,
 113428: 15,
 113429: 24,
 117147: 1,
 117148: 10,
 117149: 19,
 117209: 4,
 117210: 13,
 117211: 22,
 117271: 7,
 117272: 16,
 117273: 25,
 120991: 2,
 120992: 11,
 120993: 20,
 121053: 5,
 121054: 14,
 121055: 23,
 121115: 8,
 121116: 17,
 121117: 26}

In [29]:
groups_map.take(20)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 4 times, most recent failure: Lost task 0.3 in stage 11.0 (TID 232, 10.205.8.59): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/cluster/home/roskarr/spark/python/lib/pyspark.zip/pyspark/worker.py", line 170, in main
    profiler.profile(process)
  File "/cluster/home/roskarr/spark/python/lib/pyspark.zip/pyspark/profiler.py", line 160, in profile
    pr.runcall(func)
  File "/cluster/home/roskarr/miniconda2/lib/python2.7/cProfile.py", line 149, in runcall
    return func(*args, **kw)
  File "/cluster/home/roskarr/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/cluster/home/roskarr/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 133, in dump_stream
   

In [101]:
def pid_gid(p):
    """Map the particle to its pid and gid"""
    return (p['iOrder'], p['iGroup'])


In [ ]:
(fof_analyzer.fof_rdd.flatMap(lambda p: p[np.where(p['is_ghost'])[0]])
                             .map(lambda p: (p['pos'], p))
                             .mapValues(pid_gid)).count()

In [268]:
merged_rdd = fof_analyzer.merged_rdd.cache()
merged_rdd.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 96.0 failed 4 times, most recent failure: Lost task 3.3 in stage 96.0 (TID 342, 10.205.7.45): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/cluster/home/roskarr/spark/python/lib/pyspark.zip/pyspark/worker.py", line 170, in main
    profiler.profile(process)
  File "/cluster/home/roskarr/spark/python/lib/pyspark.zip/pyspark/profiler.py", line 160, in profile
    pr.runcall(func)
  File "/cluster/home/roskarr/miniconda2/lib/python2.7/cProfile.py", line 149, in runcall
    return func(*args, **kw)
  File "/cluster/home/roskarr/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/cluster/home/roskarr/spark/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func

In [267]:
%%time
final_fof_rdd = fof_analyzer.final_fof_rdd.cache()
#groups = fof_analyzer.groups

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 93.0 failed 4 times, most recent failure: Lost task 3.3 in stage 93.0 (TID 324, 10.205.7.45): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/cluster/home/roskarr/spark/python/lib/pyspark.zip/pyspark/worker.py", line 170, in main
    profiler.profile(process)
  File "/cluster/home/roskarr/spark/python/lib/pyspark.zip/pyspark/profiler.py", line 160, in profile
    pr.runcall(func)
  File "/cluster/home/roskarr/miniconda2/lib/python2.7/cProfile.py", line 149, in runcall
    return func(*args, **kw)
  File "/cluster/home/roskarr/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/cluster/home/roskarr/spark/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func

In [ ]:
sc.show_profiles()

In [25]:
len(groups)

1267849

In [76]:
len(groups)

2194231

In [54]:
min(groups.values())

1

In [75]:
len(fof_analyzer.group_merge_map)

880530

In [56]:
group_merge_map = fof_analyzer.group_merge_map
gr_map_inv = {v:k for (k,v) in group_merge_map.iteritems()}
gr_map_inv_b = sc.broadcast(gr_map_inv)

In [61]:
from itertools import izip
def count_groups_partition(particle_arrays, gr_map_inv_b, nMinMembers): 
    p_arr = np.concatenate(list(particle_arrays))
    gs, counts = np.unique(p_arr['iGroup'], return_counts=True)
    gr_map_inv = gr_map_inv_b.value
    return ((g,cnt) for g,cnt in izip(gs,counts) if (g in gr_map_inv) or (cnt >= nMinMembers))

In [62]:
no_ghosts_rdd = fof_analyzer.merged_rdd.map(lambda p: p[np.where(p['is_ghost'] != 2)[0]])

In [68]:
group_counts = no_ghosts_rdd.mapPartitions(lambda p_arrs: count_groups_partition(p_arrs, gr_map_inv_b, 64))

In [72]:
merge_group_counts = (group_counts.filter(lambda (g,cnt): g in gr_map_inv_b.value)
                                          .reduceByKey(lambda a,b: a+b)
                                          .filter(lambda (g,cnt): cnt>=64))


In [73]:
total_group_counts = group_counts.filter(lambda (gid,cnt): gid not in gr_map_inv_b.value) + merge_group_counts

In [74]:
total_group_counts.values().min()

64

## Used for testing

In [ ]:
def read_tipsy_output(filename, chunksize = 2048): 
    """
    Read a tipsy file and set the sequential particle IDs
    
    This scans through the data twice -- first to get partition particle counts
    and a second time to actually set the particle IDs.
    """
    
    # helper functions
    def convert_to_fof_particle_partition(index, iterator): 
        for s in iterator: 
            a = convert_to_fof_particle(s)
            if count: 
                npart_acc.add({index: len(a)})
            yield a

    def set_particle_IDs_partition(index, iterator): 
        p_counts = partition_counts.value
        local_index = 0
        start_index = sum([p_counts[i] for i in range(index)])
        for arr in iterator:
            arr['iOrder'] = range(start_index + local_index, start_index + local_index + len(arr))
            local_index += len(arr)
            yield arr
    
    rec_rdd = sc.binaryRecords(filename, pdt_tipsy.itemsize*chunksize)
    nPartitions = rec_rdd.getNumPartitions()
    # set the partition count accumulator
    npart_acc = sc.accumulator({i:0 for i in range(nPartitions)}, dictAdd())
    count=True
    # read the data and count the particles per partition
    rec_rdd = rec_rdd.mapPartitionsWithIndex(convert_to_fof_particle_partition)
    rec_rdd.count()
    count=False

    partition_counts = sc.broadcast(npart_acc.value)

    return rec_rdd.mapPartitionsWithIndex(set_particle_IDs_partition)

In [ ]:
#p_rdd = read_tipsy_output('/Users/rok/polybox/euclid256.nat_no_header')

In [ ]:
#p_rdd.cache().count()

In [ ]:
# %%time 
# nMinMembers = 1
# n_groups = fof.run(ps, tau, nMinMembers)
# print 'number of groups to %d particle = %d'%(nMinMembers, n_groups)